In [ ]:
# Install neqsim if needed (uncomment for Colab)
# !pip install neqsim

In [ ]:
from neqsim.thermo import fluid, TPflash
from neqsim import jneqsim
import matplotlib.pyplot as plt
import numpy as np

## 1. Basic Compressor Concepts

**Key parameters:**
- **Suction conditions:** P₁, T₁
- **Discharge pressure:** P₂
- **Pressure ratio:** r = P₂/P₁
- **Polytropic efficiency:** ηₚ (typically 75-85%)
- **Polytropic head:** Work per unit mass
- **Power:** Energy consumption

In [ ]:
# Create export gas mixture
gas = fluid("srk")
gas.addComponent("nitrogen", 1.0)
gas.addComponent("CO2", 0.5)
gas.addComponent("methane", 90.0)
gas.addComponent("ethane", 5.0)
gas.addComponent("propane", 2.0)
gas.addComponent("n-butane", 1.0)
gas.addComponent("n-pentane", 0.5)
gas.setMixingRule(2)

# Set inlet conditions
P_suction = 30.0  # bara
T_suction = 25.0  # °C

gas.setTemperature(T_suction, "C")
gas.setPressure(P_suction, "bara")
TPflash(gas)

print(f"Gas composition: N2=1%, CO2=0.5%, C1=90%, C2=5%, C3=2%, nC4=1%, nC5=0.5%")
print(f"\nSuction conditions:")
print(f"  Pressure: {P_suction} bara")
print(f"  Temperature: {T_suction} °C")
print(f"  Molar mass: {gas.getMolarMass() * 1000:.2f} g/mol")
print(f"  Z-factor: {gas.getZ():.4f}")

## 2. Single-Stage Compression

In [ ]:
# Create process system with compressor
from neqsim.process import stream, compressor, clearProcess

clearProcess()

# Reset gas conditions
gas.setTemperature(T_suction, "C")
gas.setPressure(P_suction, "bara")
gas.setTotalFlowRate(50.0, "MSm3/day")  # 50 MSm³/day

# Create inlet stream
inlet = stream(gas, "gas inlet")

# Create compressor
P_discharge = 90.0  # bara
comp = compressor(inlet, P_discharge, "compressor")
comp.setPolytropicEfficiency(0.78)
comp.setUsePolytropicCalc(True)

In [ ]:
# Run the process
from neqsim.process import run, getProcess
run()

# Display results
print("Single-Stage Compression Results:")
print("=" * 50)
print(f"\nInlet:")
print(f"  Pressure: {inlet.getPressure():.1f} bara")
print(f"  Temperature: {inlet.getTemperature() - 273.15:.1f} °C")

print(f"\nOutlet:")
print(f"  Pressure: {comp.getOutletStream().getPressure():.1f} bara")
print(f"  Temperature: {comp.getOutletStream().getTemperature() - 273.15:.1f} °C")

print(f"\nCompressor performance:")
print(f"  Pressure ratio: {comp.getOutletStream().getPressure() / inlet.getPressure():.2f}")
print(f"  Polytropic efficiency: {comp.getPolytropicEfficiency() * 100:.1f}%")
print(f"  Polytropic head: {comp.getPolytropicHead() / 1000:.1f} kJ/kg")
print(f"  Power: {comp.getPower() / 1e6:.2f} MW")

## 3. Effect of Polytropic Efficiency

In [ ]:
# Calculate power vs efficiency
efficiencies = np.arange(0.70, 0.90, 0.02)
powers = []
temps = []

for eff in efficiencies:
    clearProcess()
    gas.setTemperature(T_suction, "C")
    gas.setPressure(P_suction, "bara")
    gas.setTotalFlowRate(50.0, "MSm3/day")
    
    inlet = stream(gas, "gas inlet")
    comp = compressor(inlet, P_discharge, "compressor")
    comp.setPolytropicEfficiency(eff)
    comp.setUsePolytropicCalc(True)
    run()
    
    powers.append(comp.getPower() / 1e6)
    temps.append(comp.getOutletStream().getTemperature() - 273.15)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(efficiencies * 100, powers, 'b-o', linewidth=2)
ax1.set_xlabel('Polytropic Efficiency [%]', fontsize=12)
ax1.set_ylabel('Power [MW]', fontsize=12)
ax1.set_title('Compressor Power vs Efficiency', fontsize=14)
ax1.grid(True, alpha=0.3)

ax2.plot(efficiencies * 100, temps, 'r-s', linewidth=2)
ax2.set_xlabel('Polytropic Efficiency [%]', fontsize=12)
ax2.set_ylabel('Discharge Temperature [°C]', fontsize=12)
ax2.set_title('Discharge Temperature vs Efficiency', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"At 78% efficiency: Power = {powers[4]:.2f} MW, T_out = {temps[4]:.1f}°C")

## 4. Two-Stage Compression with Intercooling

For high pressure ratios, multi-stage compression with intercooling:
- Reduces discharge temperature
- Reduces power consumption
- Allows operation within material limits

In [ ]:
from neqsim.process import heater, separator

# Design two-stage compression from 30 to 200 bara
P1 = 30.0    # bara
P3 = 200.0   # bara

# Optimal intermediate pressure for equal work
P2_optimal = np.sqrt(P1 * P3)
print(f"Optimal intermediate pressure: {P2_optimal:.1f} bara")
print(f"Stage 1 ratio: {P2_optimal / P1:.2f}")
print(f"Stage 2 ratio: {P3 / P2_optimal:.2f}")

In [ ]:
# Build two-stage compression train
clearProcess()

# Fresh gas
gas.setTemperature(T_suction, "C")
gas.setPressure(P1, "bara")
gas.setTotalFlowRate(50.0, "MSm3/day")

# Process train
inlet = stream(gas, "inlet")

# Stage 1
comp1 = compressor(inlet, P2_optimal, "compressor 1")
comp1.setPolytropicEfficiency(0.78)
comp1.setUsePolytropicCalc(True)

# Intercooler (cool back to 35°C)
cooler1 = heater(comp1.getOutletStream(), "intercooler")
cooler1.setOutTemperature(35.0 + 273.15)  # K

# Stage 2
comp2 = compressor(cooler1.getOutletStream(), P3, "compressor 2")
comp2.setPolytropicEfficiency(0.78)
comp2.setUsePolytropicCalc(True)

# Run
run()

In [ ]:
# Display results
print("Two-Stage Compression Results:")
print("=" * 60)

print(f"\nStage 1 (LP Compressor):")
print(f"  Suction: {inlet.getPressure():.1f} bara, {inlet.getTemperature() - 273.15:.1f}°C")
print(f"  Discharge: {comp1.getOutletStream().getPressure():.1f} bara, {comp1.getOutletStream().getTemperature() - 273.15:.1f}°C")
print(f"  Power: {comp1.getPower() / 1e6:.2f} MW")

print(f"\nIntercooler:")
print(f"  Outlet temperature: {cooler1.getOutletStream().getTemperature() - 273.15:.1f}°C")
print(f"  Duty: {abs(cooler1.getDuty()) / 1e6:.2f} MW")

print(f"\nStage 2 (HP Compressor):")
print(f"  Suction: {cooler1.getOutletStream().getPressure():.1f} bara, {cooler1.getOutletStream().getTemperature() - 273.15:.1f}°C")
print(f"  Discharge: {comp2.getOutletStream().getPressure():.1f} bara, {comp2.getOutletStream().getTemperature() - 273.15:.1f}°C")
print(f"  Power: {comp2.getPower() / 1e6:.2f} MW")

total_power = (comp1.getPower() + comp2.getPower()) / 1e6
print(f"\nTotal compression power: {total_power:.2f} MW")

## 5. Compare Single-Stage vs Two-Stage

For the same overall pressure ratio, compare power consumption.

In [ ]:
# Single-stage from 30 to 200 bara
clearProcess()
gas.setTemperature(T_suction, "C")
gas.setPressure(P1, "bara")
gas.setTotalFlowRate(50.0, "MSm3/day")

inlet_single = stream(gas, "inlet")
comp_single = compressor(inlet_single, P3, "compressor")
comp_single.setPolytropicEfficiency(0.78)
comp_single.setUsePolytropicCalc(True)
run()

single_power = comp_single.getPower() / 1e6
single_temp = comp_single.getOutletStream().getTemperature() - 273.15

print("Comparison: Single-Stage vs Two-Stage (30 → 200 bara)")
print("=" * 60)
print(f"\n{'Parameter':<30} {'Single':<15} {'Two-Stage':<15}")
print("-" * 60)
print(f"{'Total Power [MW]':<30} {single_power:<15.2f} {total_power:<15.2f}")
print(f"{'Final Temperature [°C]':<30} {single_temp:<15.1f} {comp2.getOutletStream().getTemperature() - 273.15:<15.1f}")
print(f"{'Power Savings [%]':<30} {'-':<15} {(single_power - total_power) / single_power * 100:.1f}")

## 6. Visualizing the Compression Train

In [ ]:
# Process flow diagram (text-based)
print("""
TWO-STAGE COMPRESSION TRAIN
============================

     ┌──────────┐      ┌──────────┐      ┌──────────┐      ┌──────────┐
     │          │      │          │      │          │      │          │
──▶  │  LP      │ ──▶  │ Inter-   │ ──▶  │  HP      │ ──▶  │ After-   │ ──▶
     │ Compr.   │      │ cooler   │      │ Compr.   │      │ cooler   │
     │          │      │          │      │          │      │          │
     └──────────┘      └──────────┘      └──────────┘      └──────────┘
        Stage 1          Cooling           Stage 2          Cooling

  30 bara        77 bara       77 bara        200 bara       200 bara
  25°C          ~120°C          35°C          ~130°C          40°C
""")

## 7. Design Guidelines

In [ ]:
print("""
COMPRESSOR DESIGN GUIDELINES
============================

1. PRESSURE RATIO PER STAGE
   - Centrifugal: 1.5 - 3.0 per stage
   - Max discharge temperature: 150-180°C
   - Consider intercooling above ratio ~3

2. POLYTROPIC EFFICIENCY
   - Centrifugal: 75-82% typical
   - Reciprocating: 80-90% (isentropic)
   - Higher MW gas → higher efficiency

3. SURGE & CHOKE
   - Surge: Low flow limit (recycle needed)
   - Choke: High flow limit (sonic velocity)
   - Operating point: 10-15% above surge

4. INTERCOOLING BENEFITS
   - Reduces discharge temperature
   - Reduces power consumption (~15-25%)
   - Removes liquids between stages

5. MOLECULAR WEIGHT EFFECTS
   - Higher MW → more head per stage
   - Higher MW → higher efficiency
   - CO2-rich gas needs special consideration
""")

## Summary

This notebook covered:
- Single-stage compression calculations
- Effect of polytropic efficiency on power and temperature
- Two-stage compression with intercooling
- Power savings from staging
- Design guidelines

**Key equations:**
- Optimal intermediate pressure: $P_2 = \sqrt{P_1 \cdot P_3}$
- Polytropic head: $H_p = \int v \, dP$
- Power: $W = \dot{m} \cdot H_p / \eta_p$